In [1]:
!pip install transformers datasets peft accelerate trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requ

In [1]:
# Import necessary libraries
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer # Import the regular Trainer
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
import torch

In [2]:
# Load the TinyBERT model and tokenizer
model_name = "huawei-noah/TinyBERT_General_4L_312D"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at 

In [3]:
# Load a QA dataset (using SQuAD for this example)
dataset = load_dataset("squad")

# dataset["train"] = dataset["train"].select(range(int(len(dataset["train"]) * 0.01)))
# dataset["validation"] = dataset["validation"].select(range(int(len(dataset["validation"]) * 0.01)))


In [4]:
def answer_question(question, context, model):
    inputs = tokenizer(question, context, return_tensors="pt", truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract the start and end logits
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

    # Get the most likely start and end of the answer
    start_index = torch.argmax(start_logits)
    end_index = torch.argmax(end_logits)

    answer_tokens = inputs["input_ids"][0][start_index:end_index + 1]
    answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)
    return answer

# Define some sample questions from the dataset
sample_questions = [
    {"question": "What is the capital of France?", "context": "Paris is the capital and most populous city of France."},
    {"question": "Who wrote 'Pride and Prejudice'?", "context": "Jane Austen wrote the novel 'Pride and Prejudice'."},
    {"question": "What is the largest planet in our solar system?", "context": "Jupiter is the largest planet in the solar system."}
]

# Evaluate the model before fine-tuning
print("Answers from the model before fine-tuning:\n")
for sample in sample_questions:
    answer = answer_question(sample["question"], sample["context"], model)
    print(f"Question: {sample['question']}")
    print(f"Answer: {answer}\n")



Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Answers from the model before fine-tuning:

Question: What is the capital of France?
Answer: what is the capital of france?

Question: Who wrote 'Pride and Prejudice'?
Answer: who wrote'pride and prejudice '? jane

Question: What is the largest planet in our solar system?
Answer: 



In [5]:
# Preprocess the dataset
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        padding="max_length",
        return_tensors="pt",
        # Add this line to return offset_mapping
        return_offsets_mapping=True
    )

    # Extract offset_mapping from the inputs
    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]

    start_positions = []
    end_positions = []
    for i, offset in enumerate(offset_mapping):
        answer = answers[i]["text"][0]
        start_char = answers[i]["answer_start"][0]
        end_char = start_char + len(answer)
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end token indices in the tokenized version
        token_start_index = sequence_ids.index(1)
        token_end_index = len(sequence_ids) - 1 - sequence_ids[::-1].index(1)

        # Offset mappings: map token index to character index in the original context
        start_positions.append(token_start_index)
        end_positions.append(token_end_index)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

# Tokenize and format the dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [6]:
lora_config = LoraConfig(
    r=4,  # Rank of the decomposition matrix
    lora_alpha=16,  # Scaling factor
    lora_dropout=0.1,  # Dropout rate for LoRA
    target_modules=["query", "key"],  # LoRA applied to these layers in attention mechanism
)

In [7]:
# Apply LoRA to the model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 19,968 || all params: 14,273,186 || trainable%: 0.1399


In [8]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=False,
    save_strategy="epoch",
    # metric_for_best_model="eval_loss"  # Set the metric to monitor for best model
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions, references=labels) # Calculate and return relevant metrics

In [10]:
#  Use SFTTrainer for fine-tuning with LoRA
trainer = Trainer( #Using Trainer instead of SFTTrainer as mentioned in the code.
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)

# Fine-tune the model
trainer.train()

# Save the model
model.save_pretrained("./lora_tinybert_final")
tokenizer.save_pretrained("./lora_tinybert_final")

Epoch,Training Loss,Validation Loss
1,5.299400,No log
2,5.088100,No log


('./lora_tinybert_final/tokenizer_config.json',
 './lora_tinybert_final/special_tokens_map.json',
 './lora_tinybert_final/vocab.txt',
 './lora_tinybert_final/added_tokens.json',
 './lora_tinybert_final/tokenizer.json')

In [13]:
# Load the TinyBERT model and tokenizer
model_name = "./lora_tinybert_final"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)


print("Answers from the model before fine-tuning:\n")
for sample in sample_questions:
    answer = answer_question(sample["question"], sample["context"], model)
    print(f"Question: {sample['question']}")
    print(f"Answer: {answer}\n")



Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Answers from the model before fine-tuning:

Question: What is the capital of France?
Answer: what is the capital of france? paris is the capital and most populous city of france

Question: Who wrote 'Pride and Prejudice'?
Answer: who wrote'pride and prejudice '? jane austen wrote the novel'pride and prejudice '

Question: What is the largest planet in our solar system?
Answer: what is the largest planet in our solar system? jupiter is the largest planet in the



In [18]:
# Install necessary packages (if not already installed)
!pip install transformers datasets peft accelerate trl

# Import necessary libraries
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer # Import the regular Trainer
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
import torch


# Load the TinyBERT model and tokenizer
model_name = "huawei-noah/TinyBERT_General_4L_312D"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# Load a QA dataset (using SQuAD for this example)
dataset = load_dataset("squad")

dataset["train"] = dataset["train"].select(range(int(len(dataset["train"]) * 0.01)))
dataset["validation"] = dataset["validation"].select(range(int(len(dataset["validation"]) * 0.01)))

def answer_question(question, context, model):
    inputs = tokenizer(question, context, return_tensors="pt", truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract the start and end logits
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

    # Get the most likely start and end of the answer
    start_index = torch.argmax(start_logits)
    end_index = torch.argmax(end_logits)

    answer_tokens = inputs["input_ids"][0][start_index:end_index + 1]
    answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)
    return answer

# Define some sample questions from the dataset
sample_questions = [
    {"question": "What is the capital of France?", "context": "Paris is the capital and most populous city of France."},
    {"question": "Who wrote 'Pride and Prejudice'?", "context": "Jane Austen wrote the novel 'Pride and Prejudice'."},
    {"question": "What is the largest planet in our solar system?", "context": "Jupiter is the largest planet in the solar system."}
]

# Evaluate the model before fine-tuning
print("Answers from the model before fine-tuning:\n")
for sample in sample_questions:
    answer = answer_question(sample["question"], sample["context"], model)
    print(f"Question: {sample['question']}")
    print(f"Answer: {answer}\n")


# Preprocess the dataset
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        padding="max_length",
        return_tensors="pt",
        # Add this line to return offset_mapping
        return_offsets_mapping=True
    )

    # Extract offset_mapping from the inputs
    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]

    start_positions = []
    end_positions = []
    for i, offset in enumerate(offset_mapping):
        answer = answers[i]["text"][0]
        start_char = answers[i]["answer_start"][0]
        end_char = start_char + len(answer)
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end token indices in the tokenized version
        token_start_index = sequence_ids.index(1)
        token_end_index = len(sequence_ids) - 1 - sequence_ids[::-1].index(1)

        # Offset mappings: map token index to character index in the original context
        start_positions.append(token_start_index)
        end_positions.append(token_end_index)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

# Tokenize and format the dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Configure LoRA
lora_config = LoraConfig(
    r=2,  # Rank of the decomposition matrix
    lora_alpha=4,  # Scaling factor
    lora_dropout=0.1,  # Dropout rate for LoRA
    target_modules=["query", "key"],  # LoRA applied to these layers in attention mechanism
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=1,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=False,
    save_strategy="epoch",
    # metric_for_best_model="eval_loss"  # Set the metric to monitor for best model
)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Answers from the model before fine-tuning:

Question: What is the capital of France?
Answer: capital of france? paris is the capital and most populous city of france

Question: Who wrote 'Pride and Prejudice'?
Answer: and prejudice '? jane austen wrote the

Question: What is the largest planet in our solar system?
Answer: what is the largest planet in our solar system? jupiter is the largest planet in the solar system



Map:   0%|          | 0/105 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions, references=labels) # Calculate and return relevant metrics

In [17]:
#  Use SFTTrainer for fine-tuning with LoRA
trainer = Trainer( #Using Trainer instead of SFTTrainer as mentioned in the code.
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)

# Fine-tune the model
trainer.train()

# Save the model
model.save_pretrained("./lora_tinybert_fast")
tokenizer.save_pretrained("./lora_tinybert_fast")

Epoch,Training Loss,Validation Loss
1,5.946000,No log


('./lora_tinybert_fast/tokenizer_config.json',
 './lora_tinybert_fast/special_tokens_map.json',
 './lora_tinybert_fast/vocab.txt',
 './lora_tinybert_fast/added_tokens.json',
 './lora_tinybert_fast/tokenizer.json')

In [22]:
# Load the TinyBERT model and tokenizer
model_name = "./lora_tinybert_fast"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)


print("Answers from the model before fine-tuning:\n")
for sample in sample_questions:
    answer = answer_question(sample["question"], sample["context"], model)
    print(f"Question: {sample['question']}")
    print(f"Answer: {answer}\n")



Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Answers from the model before fine-tuning:

Question: What is the capital of France?
Answer: 

Question: Who wrote 'Pride and Prejudice'?
Answer: austen

Question: What is the largest planet in our solar system?
Answer: our solar system? jupiter is the largest

